# Filtering on the most recent version of groups in a dataframe
Recently, a friend came to me with an interesting [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) problem.  He basically had a set of survey questions and answers.  The surveys were of different software applications, but each application had one or more versions of survey results.  His goal was to filter on the most recent survey results of each application.

For demo purposes, consider a dataset different rock bands and their lineup changes over the years.  Van Halen has had about four lineup changes: from the early David Lee Roth years, to the Sammy years, to the Gary Cherone years, and back to the David Lee Roth years.  ZZ Top, on the other hand, has had only one lineup: the original three members.

The challenge: given a dataset of different bands and their different lineup changes over the years, how might you filter the dataframe to only show the most recent lineup of each band?

I found two ways to solve this problem and a third potential option that bears some consideration.  Let's take a look at these solutions!

### Of course, we must first import pandas and load our dataset

In [1]:
import pandas as pd

In [54]:
df = pd.read_csv('bands.csv')
df

,band,version,members
0,Van Halen,1,David Lee Roth
1,Van Halen,1,Eddie Van Halen
2,Van Halen,1,Alex Van Halen
3,Van Halen,1,Michael Anthony
4,Van Halen,2,Sammy Hagar
5,Van Halen,2,Eddie Van Halen
6,Van Halen,2,Alex Van Halen
7,Van Halen,2,Michael Anthony
8,Van Halen,3,Gary Cherone
9,Van Halen,3,Eddie Van Halen


### Solution 1: idxmax and Merge
The [idxmax](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.idxmax.html) function returns the row locations where a given column contains the maximum value.  With our dataset, then, we can group by the *band* then run idxmax on the *version* column.  Unfortunately, though, idxmax only returns one row per band; otherwise, it might have solved our problem straight-away:

In [8]:
# idxmax won't work because it only returns one record per group
df.loc[df.groupby('band').version.idxmax()]

,band,version,members
32,Kiss,5,Paul Stanley
36,Rush,2,Geddy Lee
12,Van Halen,4,David Lee Roth
42,ZZ Top,1,Billy Gibbons


#### Now Merge
With idxmax, we can still build a "most recent" dataframe: as in the most recent version numbers for each band.  Then, we can use pandas [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) to join our full dataset to the one representing the most recent versions of each band.  This will effectively filter down our full dataframe to one that only shows the most recent versions of each band and, thus, solve our problem:

In [21]:
df_most_recent = df.loc[df.groupby('band').version.idxmax()]
df.merge(df_most_recent, how='right', on=['band', 'version'])

,band,version,members_x,members_y
0,Van Halen,4,David Lee Roth,David Lee Roth
1,Van Halen,4,Eddie Van Halen,David Lee Roth
2,Van Halen,4,Alex Van Halen,David Lee Roth
3,Van Halen,4,Wolfgang Van Halen,David Lee Roth
4,Kiss,5,Paul Stanley,Paul Stanley
5,Kiss,5,Gene Simmons,Paul Stanley
6,Kiss,5,Tommy Thayer,Paul Stanley
7,Kiss,5,Eric Singer,Paul Stanley
8,Rush,2,Geddy Lee,Geddy Lee
9,Rush,2,Alex Lifeson,Geddy Lee


<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>
### But, wait, there's more!

### Max and clever filtering
Similar to idxmax, we can use the pandas [max](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.max.html) function to also find the latest version numbers of each band, but this time as a [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html).

In [12]:
df.groupby('band').version.max()

band
Kiss         5
Rush         2
Van Halen    4
ZZ Top       1
Name: version, dtype: int64

Series objects can be easily casted to a [dictionary](https://www.w3schools.com/python/python_dictionaries.asp) object:

In [37]:
dict(most_recent).items()

dict_items([('Kiss', 5), ('Rush', 2), ('Van Halen', 4), ('ZZ Top', 1)])

This next part is really cool: I was not aware you could do this, but you can use the [apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) to convert dataframe columns into a Series of [tuples](https://www.w3schools.com/python/python_tuples.asp).  Then, you can use the [isin](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) function and some clever [list comprehension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python) to match the band/version tuple of the main dataframe to the band/version tuple of the "most recent" Series.  The result is the solution we seek: a dataframe of the most recent band lineups:

In [50]:
# https://stackoverflow.com/questions/53945870/filter-pandas-dataframe-based-on-combination-of-two-columns
most_recent = dict(df.groupby('band').version.max())
df[ (df[['band','version']].apply(tuple, axis=1)).isin([(b,v) for b, v in most_recent.items()]) ]

,band,version,members
12,Van Halen,4,David Lee Roth
13,Van Halen,4,Eddie Van Halen
14,Van Halen,4,Alex Van Halen
15,Van Halen,4,Wolfgang Van Halen
32,Kiss,5,Paul Stanley
33,Kiss,5,Gene Simmons
34,Kiss,5,Tommy Thayer
35,Kiss,5,Eric Singer
36,Rush,2,Geddy Lee
37,Rush,2,Alex Lifeson


<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>
### Another consideration
So, the above two approaches seem to solve this problem decently.  One other option to explore is changing the index of the dataframe to a multi-index dataframe on the band and lineup version:

In [24]:
df_indexed = df.set_index(['band', 'version'])
df_indexed

members
band      version                    
Van Halen 1            David Lee Roth
          1           Eddie Van Halen
          1            Alex Van Halen
          1           Michael Anthony
          2               Sammy Hagar
          2           Eddie Van Halen
          2            Alex Van Halen
          2           Michael Anthony
          3              Gary Cherone
          3           Eddie Van Halen
          3            Alex Van Halen
          3           Michael Anthony
          4            David Lee Roth
          4           Eddie Van Halen
          4            Alex Van Halen
          4        Wolfgang Van Halen
Kiss      1              Paul Stanley
          1              Gene Simmons
          1               Ace Frehley
          1               Peter Criss
          2              Paul Stanley
          2              Gene Simmons
          2               Ace Frehley
          2                 Eric Carr
          3              Paul Stanley
          3              Gene Simmons
          3            Vinnie Vincent
          3                 Eric Carr
          4              Paul Stanley
          4              Gene Simmons
          4              Bruce Kulick
          4                 Eric Carr
          5              Paul Stanley
          5              Gene Simmons
          5              Tommy Thayer
          5               Eric Singer
Rush      2                 Geddy Lee
          2              Alex Lifeson
          2                Neil Peart
          1                 Geddy Lee
          1              Alex Lifeson
          1               John Rutsey
ZZ Top    1             Billy Gibbons
          1                Dusty Hill
          1               Frank Beard

Pandas can do some interesting slicing based on indices, including making use of the ["cross section" (xs)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html) function.  In my example below, I'm filtering the dataframe on the second index and requesting all rows where the second index has a value of "1".  Basically, give me a dataframe of all the original band lineups:

In [26]:
df_indexed.xs(1, level=1, drop_level=False)

members
band      version                 
Van Halen 1         David Lee Roth
          1        Eddie Van Halen
          1         Alex Van Halen
          1        Michael Anthony
Kiss      1           Paul Stanley
          1           Gene Simmons
          1            Ace Frehley
          1            Peter Criss
Rush      1              Geddy Lee
          1           Alex Lifeson
          1            John Rutsey
ZZ Top    1          Billy Gibbons
          1             Dusty Hill
          1            Frank Beard

Pretty slick.  Now, how do I use this multi-index approach to return the most recent band lineups.  Unfortunately, I have not found a way to do that with the *xs* function.  Nevertheless, it would not surprise me if such a way were possible with indexing and multi-indexing.